2018-07-06

In [1]:
from  pymatgen.ext.matproj  import MPRester#Materials Project REST API的适配器
from pymatgen import Composition#可以非常灵活的处理化学元素
from pymatgen.entries.computed_entries import ComputedEntry#从数据获取的数据生成的ComputedEntries列表，主要用于管理数据的
from pymatgen.core.units import FloatWithUnit#相关化学计算的单位转换功能
from pymatgen.analysis.reaction_calculator import ComputedReaction#计算化学方程式反应的能量的

在Pymatgen中我们可以获得一个强大的工具——Materials Project REST API的适配器。引入Materials Project REST API（简称Materials API），为用户提供了一种以编程方式查询材料数据的方法。这是一个用户友好的Materials API高级接口，用于获取有用的pymatgen对象以供进一步分析。要使用Materials API，您需要首先注册Materials Project，并在在网站中可以获取API秘钥。有了API秘钥我们可以得到一个化学式的所有相关信息（Pymatgen中称之为条目-entries），并根据得到这些entries做出许多复杂的分析。

In [3]:
a = MPRester("S3lw1QomLO8bTiT7")#这将初始化REST适配器，并且将API秘钥加入其中，可以从MP网站中获取到数据
all_entries = a.get_entries_in_chemsys(['Ca', 'C', 'O'])#This gets all entries belonging to the Ca-C-O system.

In [4]:
def get_most_stable_entry(formula):
    relevant_entries = [entry for entry in all_entries if entry.composition.reduced_formula == Composition(formula).reduced_formula]
    relevant_entries = sorted(relevant_entries, key=lambda e: e.energy_per_atom)
    return relevant_entries[0]

In [5]:
CaO = get_most_stable_entry("CaO")

In [6]:
CO2 = get_most_stable_entry("CO2")
CaCO3 = get_most_stable_entry("CaCO3")

In [7]:
reaction = ComputedReaction([CaO, CO2], [CaCO3])

用于从ComputedEntry对象生成反应的便捷类。
ComputedReaction(反应物,生成物) 最后得到化学反应。

In [8]:
energy = FloatWithUnit(reaction.calculated_reaction_energy, "eV atom^-1")

In [9]:
print("Caculated")
print(reaction)
print("Reaction energy = {}".format(energy.to("kJ mol^-1")))

Caculated
CaO + CO2 -> CaCO3
Reaction energy = -146.92310939262524 kJ mol^-1


下面的部分演示了如何获得实验值

In [10]:
# The following portions demonstrate how to get the experimental values as well.
exp_CaO = a.get_exp_entry("CaO")
exp_CaCO3 = a.get_exp_entry("CaCO3")

返回ExpEntry对象，它是ComputedEntry的实验等效对象，可用于使用实验数据进行分析。
get_exp_entry(formula)：
    Parameters:formula-化学式
    Returns:一个ExpEntry对象

In [12]:
#这是一个气相实验，而且数据库没有关于这个的数据条目，所以我们要自己创建。NIST是化学信息的标准参考数据库。这里的相关数据来自NIST
exp_CO2 = ComputedEntry("CO2", -393.51)#EXP条目应为KJ/mol。
exp_reaction = ComputedReaction([exp_CaO, exp_CO2], [exp_CaCO3])

In [13]:
print("Experimental")
print(exp_reaction)
print("Reaction energy = {} kJ mol^-1".format(exp_reaction.calculated_reaction_energy))

Experimental
CaO + CO2 -> CaCO3
Reaction energy = -178.30000000000018 kJ mol^-1
